In [71]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [72]:
# from sklearn.model_selection import train_test_split    
df1 = pd.read_csv('main_dataset_kNN.csv')
df2 = pd.read_csv('drug_dict_onehot.csv')
df1.head()


,Unnamed: 0,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,...,0.368876,0.073375,0.944444,0.400000,0.945187,0.420988,0.428889,0.408730,1.0,0.666667
1,1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440000,0.226190,1.0,0.533333
2,2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440000,0.410714,1.0,0.800000
3,3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,...,0.372375,0.087002,0.917778,0.335225,0.918449,0.435802,0.473333,0.549603,1.0,0.666667
4,4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,...,0.435477,0.084906,0.965185,0.412057,0.968731,0.386265,0.473333,0.581349,1.0,0.266667


In [73]:
# df1 = df1.sort_values(by=['inpatient.number'], ascending=True)
# df1.reset_index(drop = True, inplace = True)
# df1.head()

In [74]:
df2 = df2.sort_values(by=['inpatient.number'], ascending=True)
df2.reset_index(drop = True, inplace = True)
df2.head()

,Unnamed: 0,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [75]:
#pd.concat([df1['inpatient.number'],df2['inpatient.number']]).drop_duplicates(keep=False)

In [76]:
#df1['inpatient.number'].equals(df2['inpatient.number'])

In [77]:
df1.drop(['Unnamed: 0'],axis= 1,inplace=True)
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,0.368876,0.073375,0.944444,0.400000,0.945187,0.420988,0.428889,0.408730,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440000,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440000,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,0.372375,0.087002,0.917778,0.335225,0.918449,0.435802,0.473333,0.549603,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,0.435477,0.084906,0.965185,0.412057,0.968731,0.386265,0.473333,0.581349,1.0,0.266667


In [78]:
df2.drop(['inpatient.number','Unnamed: 0'],axis= 1,inplace=True)
df2.head()
len(df2.columns)

18

In [79]:
X = df1
y = df2


In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
len(X), len(y_train), len(y_test)

(1996, 1596, 400)

In [148]:
model = Sequential()
model.add(Dense(30, input_dim=144, activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))


In [149]:
model.fit(X_train, y_train, epochs = 100, batch_size = 512)


Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2397
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1805
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1616
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1550
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1531
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1527
Epoch 7/100
4/4 [==============================] - 0s 9ms/step - loss: 0.1507
Epoch 8/100
4/4 [==============================] - 0s 6ms/step - loss: 0.1512
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 0.1507
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 0.1504
Epoch 11/100
4/4 [==============================] - 0s 4ms/step - loss: 0.1499
Epoch 12/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1497
Epoch 13/100
4/4 [==============================] - 0s 3ms/st

In [150]:
model.evaluate(X_test,y_test)

13/13 [==============================] - 0s 914us/step - loss: 0.1487


0.14866214990615845

In [151]:
X_test

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
889,0.0,0.185714,0.297980,0.500000,0.603175,0.342466,0.463235,0.036053,1.0,1.0,...,0.421479,0.098532,0.962963,0.460757,0.964424,0.396451,0.438889,0.570437,1.0,0.800000
1674,1.0,0.214286,0.247475,0.527778,0.476190,0.575342,0.529412,0.049443,0.0,0.5,...,0.401575,0.283019,0.960000,0.302128,0.942513,0.441667,0.600000,0.744048,1.0,0.666667
414,1.0,0.142857,0.368687,0.527778,0.555556,0.575342,0.566176,0.061293,1.0,0.5,...,0.399606,0.056604,0.960000,0.272340,0.962567,0.352778,0.500000,0.666667,1.0,0.533333
1427,0.0,0.185714,0.328283,0.527778,0.420635,0.575342,0.503676,0.066410,1.0,0.5,...,0.391732,0.056604,0.986667,0.604255,0.990642,0.580556,0.500000,0.607143,1.0,0.666667
849,0.0,0.285714,0.444444,0.500000,0.428571,0.369863,0.397059,0.046195,1.0,0.5,...,0.318898,0.082809,0.948148,0.418203,0.951203,0.445525,0.424444,0.418651,1.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1.0,0.185714,0.464646,0.555556,0.484127,0.561644,0.525735,0.039087,1.0,0.5,...,0.368110,0.245283,0.973333,0.336170,0.958556,0.252778,0.400000,0.821429,1.0,0.533333
576,1.0,0.142857,0.388889,0.611111,0.674603,0.753425,0.716912,0.058002,0.5,1.0,...,0.387795,0.094340,0.986667,0.685106,0.989305,0.422222,0.500000,0.696429,1.0,0.533333
963,1.0,0.185714,0.303030,0.527778,0.515873,0.547945,0.533088,0.064224,1.0,0.5,...,0.377187,0.085954,0.961481,0.480851,0.963458,0.385648,0.447778,0.520172,1.0,0.800000
240,0.0,0.200000,0.333333,0.555556,0.567460,0.547945,0.556985,0.050162,1.0,0.5,...,0.386920,0.092243,0.938519,0.325059,0.940285,0.451852,0.482222,0.511574,1.0,0.533333


In [152]:
a = model.predict(X_test)
a[0]

array([0.32701254, 0.20763007, 0.8057507 , 0.84023494, 0.00336581,
       0.98939955, 0.96304107, 0.02571312, 0.00768825, 0.07958937,
       0.63034785, 0.71764493, 0.02252761, 0.2250649 , 0.9926312 ,
       0.13995987, 0.18464461, 0.32592297], dtype=float32)

In [153]:
m = []
s = []
for x in a[0]:
    m.append(round(x))
    s.append(x)
print(m,s)

[0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0] [0.32701254, 0.20763007, 0.8057507, 0.84023494, 0.0033658147, 0.98939955, 0.96304107, 0.025713116, 0.007688254, 0.07958937, 0.63034785, 0.71764493, 0.022527605, 0.2250649, 0.9926312, 0.13995987, 0.18464461, 0.32592297]


In [154]:
y_test.head()

,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
889,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0
1674,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1
414,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,1
1427,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0
849,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
